In [1]:
import pandas as pd
import json


In [2]:
df=pd.read_csv('../resources/loans_final_state.csv')
df.head()

,Unnamed: 0,loan_type,loan_amount_000s,action_taken,state_code,applicant_ethnicity,co_applicant_ethnicity,applicant_race_1,co_applicant_race_1,applicant_sex,co_applicant_sex,applicant_income_000s
0,1,1,144,1,2,2,2,5,5,1,2,154
1,2,1,300,1,2,2,5,5,8,1,5,119
2,7,1,263,1,2,1,5,5,8,1,5,141
3,28,1,187,1,2,2,5,5,8,1,5,76
4,35,1,361,1,2,2,2,1,5,2,1,107


In [3]:
df=df.drop(columns=['Unnamed: 0'])

In [4]:
# df['state_code'].value_counts()

In [5]:
# drop outlier code 78 which is not a FIPS code
df=df[df['state_code'] != 78]

In [6]:
# Get Averages (by State) for the numerical values
avg_loan = df.groupby('state_code')['loan_amount_000s'].mean()
avg_inc = df.groupby('state_code')['applicant_income_000s'].mean()
loan_ct= df.groupby('state_code')['loan_amount_000s'].count()

In [7]:
#Create new DF that has the grouped_by money data
money=pd.DataFrame({'Average_Loan_000s': avg_loan, 'Average_Income_000s': avg_inc, 'Total_Loans': loan_ct}).reset_index()


In [8]:
# First, Get a count of each response (by state) for each of the the categorical columns
action_taken_ct=df.groupby('state_code')['action_taken'].value_counts().unstack().reset_index()
loan_type_ct=df.groupby('state_code')['loan_type'].value_counts().unstack().reset_index()
app_eth_ct=df.groupby('state_code')['applicant_ethnicity'].value_counts().unstack().reset_index()
co_app_eth_ct=df.groupby('state_code')['co_applicant_ethnicity'].value_counts().unstack().reset_index()
app_race_ct=df.groupby('state_code')['applicant_race_1'].value_counts().unstack().reset_index()
co_app_race_ct=df.groupby('state_code')['co_applicant_race_1'].value_counts().unstack().reset_index()
app_sex_ct=df.groupby('state_code')['applicant_sex'].value_counts().unstack().reset_index()
co_app_sex_ct=df.groupby('state_code')['co_applicant_sex'].value_counts().unstack().reset_index()

In [9]:
#Rename each of those numerical categorizations to their actual value
# This way they they will show in dict as keys 
action_taken_ct=action_taken_ct.rename(columns={1:'Qualified', 0:'Failed_to_Qualify'})
loan_type_ct=loan_type_ct.rename(columns={1:'Conventional', 2:'FHA_Insured' , 3:'VA-guaranteed' , 4: 'FSA/RHS' })
app_eth_ct=app_eth_ct.rename(columns={1: 'Applicant_Hispanic_or_Latino', 2: 'Applicant_Not_Hispanic_or_Latino'})
co_app_eth_ct=co_app_eth_ct.rename(columns={1: 'Co-Applicant_Hispanic_or_Latino', 2: 'Co-Applicant_Not_Hispanic_or_Latino', 5: 'No_co-applicant_(Ethnicity)'})
app_race_ct=app_race_ct.rename(columns={1:'Applicant_American_Indian_or_Alaska_Native', 2: 'Applicant_Asian', 3:'Applicant_Black_or_African_American', 4: 'Applicant_Native_Hawaiian_or_Other_Pacific_Islander', 5:'Applicant_White'})
co_app_race_ct=co_app_race_ct.rename(columns={1:'Co-Applicant_American_Indian_or_Alaska_Native', 2: 'Co-Applicant_Asian', 3:'Co-Applicant_Black_or_African_American', 4: 'Co-Applicant_Native_Hawaiian_or_Other_Pacific_Islander', 5:'Co-Applicant_White', 8: 'No_Co-applicant_(Race)'})
app_sex_ct=app_sex_ct.rename(columns={1:'Applicant_Male', 2:'Applicant_Female'})
co_app_sex_ct=co_app_sex_ct.rename(columns={1:'Co-Applicant_Male', 2:'Co-Applicant_Female', 5:'No_Co-applicant_(Gender)'})

In [10]:
# Merging Counts 
eth=app_eth_ct.merge(co_app_eth_ct, left_on="state_code", right_on="state_code")
sex=app_sex_ct.merge(co_app_sex_ct, left_on="state_code", right_on="state_code")
race=co_app_race_ct.merge(app_race_ct, left_on="state_code", right_on="state_code")
ltype=loan_type_ct.merge(action_taken_ct, left_on="state_code", right_on="state_code")



In [11]:
# Merging the merges
alleth=eth.merge(sex, left_on="state_code", right_on="state_code")
allrace=ltype.merge(race, left_on="state_code", right_on="state_code")


In [12]:
# Merging the merges part 2
almost=alleth.merge(allrace, left_on="state_code", right_on="state_code")

In [13]:
# Adding in the money DF created above
states=almost.merge(money, left_on="state_code", right_on="state_code")


In [14]:
#Creating a 52nd "State" which is the sum of all the values to give national data
states.loc['52']= states.sum()

In [15]:
states.head()

,state_code,Applicant_Hispanic_or_Latino,Applicant_Not_Hispanic_or_Latino,Co-Applicant_Hispanic_or_Latino,Co-Applicant_Not_Hispanic_or_Latino,No_co-applicant_(Ethnicity),Applicant_Male,Applicant_Female,Co-Applicant_Male,Co-Applicant_Female,...,Co-Applicant_White,No_Co-applicant_(Race),Applicant_American_Indian_or_Alaska_Native,Applicant_Asian,Applicant_Black_or_African_American,Applicant_Native_Hawaiian_or_Other_Pacific_Islander,Applicant_White,Average_Loan_000s,Average_Income_000s,Total_Loans
0,1.0,5180.0,202796.0,1651.0,75424.0,130901.0,146112.0,61864.0,13339.0,63736.0,...,69328.0,130901.0,1090.0,4339.0,31064.0,455.0,171028.0,170.736585,86.018050,207976.0
1,2.0,1476.0,34289.0,659.0,14726.0,20380.0,25067.0,10698.0,3564.0,11821.0,...,13205.0,20380.0,2958.0,1715.0,833.0,374.0,29885.0,273.728142,103.963400,35765.0
2,4.0,79874.0,340198.0,30778.0,166966.0,222328.0,295953.0,124119.0,35286.0,162458.0,...,184002.0,222328.0,3563.0,17110.0,12661.0,1910.0,384828.0,226.896806,98.987883,420072.0
3,5.0,7847.0,128982.0,2584.0,60314.0,73931.0,97800.0,39029.0,9816.0,53082.0,...,58718.0,73931.0,991.0,2878.0,10759.0,337.0,121864.0,150.859365,86.124674,136829.0
4,6.0,309361.0,1048942.0,151813.0,525278.0,681212.0,986085.0,372218.0,133451.0,543640.0,...,515444.0,681212.0,11741.0,267598.0,52824.0,13732.0,1012408.0,424.091164,148.832881,1358303.0


In [16]:
states['state_code'] = states['state_code'].astype(int)

In [17]:
states.dtypes

state_code                                                  int64
Applicant_Hispanic_or_Latino                              float64
Applicant_Not_Hispanic_or_Latino                          float64
Co-Applicant_Hispanic_or_Latino                           float64
Co-Applicant_Not_Hispanic_or_Latino                       float64
No_co-applicant_(Ethnicity)                               float64
Applicant_Male                                            float64
Applicant_Female                                          float64
Co-Applicant_Male                                         float64
Co-Applicant_Female                                       float64
No_Co-applicant_(Gender)                                  float64
Conventional                                              float64
FHA_Insured                                               float64
VA-guaranteed                                             float64
FSA/RHS                                                   float64
Failed_to_

In [18]:
# #set state code to str for looping to create json
states['state_code'] = states['state_code'].astype(str)
numeric_col=states.columns.drop('state_code')
states[numeric_col]=states[numeric_col].astype(int)

In [19]:
states.dtypes

state_code                                                object
Applicant_Hispanic_or_Latino                               int64
Applicant_Not_Hispanic_or_Latino                           int64
Co-Applicant_Hispanic_or_Latino                            int64
Co-Applicant_Not_Hispanic_or_Latino                        int64
No_co-applicant_(Ethnicity)                                int64
Applicant_Male                                             int64
Applicant_Female                                           int64
Co-Applicant_Male                                          int64
Co-Applicant_Female                                        int64
No_Co-applicant_(Gender)                                   int64
Conventional                                               int64
FHA_Insured                                                int64
VA-guaranteed                                              int64
FSA/RHS                                                    int64
Failed_to_Qualify        

In [20]:
states

,state_code,Applicant_Hispanic_or_Latino,Applicant_Not_Hispanic_or_Latino,Co-Applicant_Hispanic_or_Latino,Co-Applicant_Not_Hispanic_or_Latino,No_co-applicant_(Ethnicity),Applicant_Male,Applicant_Female,Co-Applicant_Male,Co-Applicant_Female,...,Co-Applicant_White,No_Co-applicant_(Race),Applicant_American_Indian_or_Alaska_Native,Applicant_Asian,Applicant_Black_or_African_American,Applicant_Native_Hawaiian_or_Other_Pacific_Islander,Applicant_White,Average_Loan_000s,Average_Income_000s,Total_Loans
0,1,5180,202796,1651,75424,130901,146112,61864,13339,63736,...,69328,130901,1090,4339,31064,455,171028,170,86,207976
1,2,1476,34289,659,14726,20380,25067,10698,3564,11821,...,13205,20380,2958,1715,833,374,29885,273,103,35765
2,4,79874,340198,30778,166966,222328,295953,124119,35286,162458,...,184002,222328,3563,17110,12661,1910,384828,226,98,420072
3,5,7847,128982,2584,60314,73931,97800,39029,9816,53082,...,58718,73931,991,2878,10759,337,121864,150,86,136829
4,6,309361,1048942,151813,525278,681212,986085,372218,133451,543640,...,515444,681212,11741,267598,52824,13732,1012408,424,148,1358303
5,8,42472,360378,17116,174149,211585,281220,121630,42193,149072,...,178333,211585,2746,17018,10529,1437,371120,289,115,402850
6,9,13674,128170,4721,54609,82514,94452,47392,14402,44928,...,53195,82514,448,7631,10539,444,122782,275,127,141844
7,10,2565,47715,853,21434,27993,33237,17043,4724,17563,...,19330,27993,184,2465,7870,139,39622,240,107,50280
8,11,1670,23978,674,8993,15981,15015,10633,3270,6397,...,7897,15981,123,2159,5034,70,18262,479,186,25648
9,12,233632,894759,84420,380293,663678,771904,356487,101566,363147,...,417346,663678,4850,39105,101409,4101,978926,227,112,1128391


### Add In State Name for JSON

In [21]:
codes=pd.read_csv("../resources/st_county_fips-v2017.csv")
codes=codes.astype(str)

In [22]:
codes.dtypes

State    object
Name     object
dtype: object

In [23]:
loanwname=states.merge(codes, left_on='state_code', right_on='State')
loanwname

,state_code,Applicant_Hispanic_or_Latino,Applicant_Not_Hispanic_or_Latino,Co-Applicant_Hispanic_or_Latino,Co-Applicant_Not_Hispanic_or_Latino,No_co-applicant_(Ethnicity),Applicant_Male,Applicant_Female,Co-Applicant_Male,Co-Applicant_Female,...,Applicant_American_Indian_or_Alaska_Native,Applicant_Asian,Applicant_Black_or_African_American,Applicant_Native_Hawaiian_or_Other_Pacific_Islander,Applicant_White,Average_Loan_000s,Average_Income_000s,Total_Loans,State,Name
0,1,5180,202796,1651,75424,130901,146112,61864,13339,63736,...,1090,4339,31064,455,171028,170,86,207976,1,Alabama
1,2,1476,34289,659,14726,20380,25067,10698,3564,11821,...,2958,1715,833,374,29885,273,103,35765,2,Alaska
2,4,79874,340198,30778,166966,222328,295953,124119,35286,162458,...,3563,17110,12661,1910,384828,226,98,420072,4,Arizona
3,5,7847,128982,2584,60314,73931,97800,39029,9816,53082,...,991,2878,10759,337,121864,150,86,136829,5,Arkansas
4,6,309361,1048942,151813,525278,681212,986085,372218,133451,543640,...,11741,267598,52824,13732,1012408,424,148,1358303,6,California
5,8,42472,360378,17116,174149,211585,281220,121630,42193,149072,...,2746,17018,10529,1437,371120,289,115,402850,8,Colorado
6,9,13674,128170,4721,54609,82514,94452,47392,14402,44928,...,448,7631,10539,444,122782,275,127,141844,9,Connecticut
7,10,2565,47715,853,21434,27993,33237,17043,4724,17563,...,184,2465,7870,139,39622,240,107,50280,10,Delaware
8,11,1670,23978,674,8993,15981,15015,10633,3270,6397,...,123,2159,5034,70,18262,479,186,25648,11,District of Columbia
9,12,233632,894759,84420,380293,663678,771904,356487,101566,363147,...,4850,39105,101409,4101,978926,227,112,1128391,12,Florida


In [24]:
loanwname.to_json('../resources/state_summary_remix.json', orient = 'records')

In [25]:
loanwname.to_csv('../resources/state_summary_remix.csv')